In [1]:
import mechanize
import http.cookiejar
from bs4 import BeautifulSoup
import re

import pandas as pd
import time

In [2]:
# Browser
br = mechanize.Browser()

# Cookie Jar
cj = http.cookiejar.LWPCookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]

# Load Article URLs
Edit this to reload article urls, remove duplicates

In [17]:
# Load old articles to compare against
old_articles = pd.read_excel('article_urls.xlsx', engine='openpyxl')
old_articles = old_articles[0]

In [5]:
start_time = time.time()

# List to save urls
articles = set()

for page in range(1,200): #46790
    
    # URL to load
    url = 'https://news.mn/archive/page/' + str(page) + '/'

    # Loaded html
    html = br.open(url).read()

    # Read html with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    # Find and print title to check we have loaded correctly
    print('Page ' + str(page) + ': ' + str(time.time() - start_time))
    
    # Find article urls
    for element in soup.find_all('article'):
        articles.add(element.a['href'])

# Save urls as pandas series
articles = pd.Series(list(articles))

print(time.time() - start_time)

Page 30: 0.7179417610168457
Page 31: 1.1474525928497314
Page 32: 1.573232889175415
Page 33: 2.026923179626465
Page 34: 2.5510408878326416
Page 35: 2.964362621307373
Page 36: 3.4058287143707275
Page 37: 3.9119763374328613
Page 38: 4.432803392410278
Page 39: 4.948277711868286
Page 40: 5.49561333656311
Page 41: 6.014770030975342
Page 42: 6.440875768661499
Page 43: 6.890276670455933
Page 44: 7.398608446121216
Page 45: 8.08446455001831
Page 46: 8.633546113967896
Page 47: 9.005419731140137
Page 48: 9.42855954170227
Page 49: 9.913002729415894
Page 50: 10.526365518569946
Page 51: 10.978309869766235
Page 52: 11.407224416732788
Page 53: 14.401704549789429
Page 54: 17.970412254333496
Page 55: 21.878312826156616
Page 56: 25.351075172424316
Page 57: 28.54083776473999
Page 58: 31.89612579345703
Page 59: 35.55154275894165
Page 60: 38.728839635849
Page 61: 42.18715000152588
Page 62: 45.540457010269165
Page 63: 48.88914394378662
Page 64: 51.31832480430603
Page 65: 53.98064875602722
Page 66: 56.23933625

TypeError: 'set' type is unordered

In [20]:
# Find articles we haven't scraped yet
unread_articles = articles[~articles.isin(old_articles)]

# Add articles to list of old articles and save
read_articles = old_articles.append(unread_articles)
read_articles.to_excel('article_urls.xlsx', index=False, header=False)

# Extract Text from Articles

In [67]:
# Load current frequency list
frequency_list = pd.read_excel('frequency_list_sample.xlsx', engine='openpyxl')
frequency_list = frequency_list.set_index('Word')

In [72]:
start_time = time.time()

# Mongolian alphabet
tolgoi = '[^АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоӨөПпРрСсТтУуҮүФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя ]'

# Pandas series to save words in
word_list = pd.Series(dtype='object')
total_text = ""

for url, count in zip(unread_articles[:10], range(len(unread_articles[:10]))):

    # Loaded html
    html = br.open(url).read()

    # Read html with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    # Find and print title to check we have loaded correctly
    print('Article ' + str(count) + ': ' + soup.find('title').string)
    
    # Extract text from the article
    text = soup.find('div', class_='has-content-area').get_text()
    total_text += " " + text
    
    article_word_list = pd.Series(re.sub(tolgoi, ' ', text).lower().split())
    word_list = word_list.append(article_word_list, ignore_index=True)

new_word_frequency = pd.DataFrame({'Count': word_list.value_counts()})
new_word_frequency.index.rename('Word', inplace=True)

print(time.time() - start_time)

Article 0: “Оюутны хөгжлийн зээлийн санхүүжилтийг ирэх сард олгоно” | News.MN
Article 1: Н.Түвшинбаярыг 20 хоног хорино | News.MN
Article 2: Япон улс Байденыг Олимпийн наадамд урихаар төлөвлөжээ | News.MN
Article 3: "Хавдартай хүн халдвар авбал нас барах эрсдэл маш өндөр" | News.MN
Article 4: Тариаланчдыг өнөөдрөөс эхлэн хоёр өдөр вакцинжуулна | News.MN
Article 5: Ё.Отгонбаярыг Элчин сайдаас нь эгүүлэн татна & татахгүй | News.MN
Article 6: Хятад улс энхийг сахиулагчдад 300 мянган тун вакцин хандивлана | News.MN
Article 7: “Сибирийн хүч-2” төслийн Монголд өгөх өгөөж | News.MN
Article 8: E-Mongolia систем 1 сая хэрэглэгчтэй боллоо | News.MN
Article 9: Эдийн засаг сэргэж буй энэ үед ААН-үүдэд зөвлөж ажиллахыг санууллаа | News.MN
8.466898202896118


In [69]:
# Add new word counts to frequency list
new_frequency_list = frequency_list.add(new_word_frequency, fill_value=0).sort_values('Count', ascending=False)

In [71]:
# Save updated frequency list
new_frequency_list.to_excel('frequency_list.xlsx')

# Search for example sentences

In [75]:
re.findall('байна', total_text)

['байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна',
 'байна']

# Sandbox

In [62]:
a = pd.DataFrame({'Count': word_list.value_counts()})
a

,Count
нь,9658
байна,7354
байгаа,4137
энэ,3731
улсын,3189
...,...
мэндлэв,1
дургүйцлээ,1
хонуулаад,1
нийлүүлэгчтэй,1


In [61]:
b = frequency_list.set_index('Word')
b

,Count
Word,
нь,1857
байна,1632
байгаа,822
энэ,784
улсын,776
...,...
хувьаймгууд,1
бүсийг,1
нэ,1


In [3]:
a = pd.DataFrame({'test': ['a', 'B']})

In [7]:
a.test.str.lower()

0    a
1    b
Name: test, dtype: object